In [3]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import json

import regex as re
from python_proj.utils.util import safe_save_fig, subtract_dict, Counter
import python_proj.utils.exp_utils as exp_utils

file_name = "dataset_90_days_started_11_07_23_preprocessed"
base_path = exp_utils.BASE_PATH
data_path = f'{base_path}/final_data/{file_name}.csv'
figure_base_path = f"{base_path}/figures/modelling/{file_name}/"

# Meta stuff.
pr_merged_key = 'PullRequestIsMerged'
ftc_key = 'SubmitterIsFirstTimeContributor'
seed_counter = Counter()


# Loads regular dataframe.
df: pd.DataFrame = pd.read_csv(filepath_or_buffer=data_path, header=0)
if 'Unnamed: 1' in df.columns:
    df = df.drop(['Unnamed: 1', "Project Name.1"], axis=1)
print(df.columns)

# Creates sub-datasets.
metadata_fields = ['ID', 'Project Name',
                   'Submitter ID', 'PR Number', 'Closed At']
dependent_fields = [pr_merged_key]
independent_fields = [field for field in df.columns
                      if (field not in metadata_fields
                          and field != pr_merged_key)]

df[independent_fields].describe()

class_counts = df[pr_merged_key].value_counts()
class_imbalance = class_counts[True] / class_counts[False]
print(f'{class_imbalance=}')

meta_header_count = 5

df.describe()

Index(['Project Name', 'ID', 'Submitter ID', 'PR Number', 'Closed At',
       'PullRequestIsMerged', 'ControlIntegratedBySameUser',
       'ControlPullRequestHasComments', 'ControlHasHashTagInDescription',
       'IntraProjectSubmitterPullRequestSuccessRate',
       'EcosystemExperienceSubmitterPullRequestSuccessRate',
       'DependencyEcosystemExperienceSubmitterPullRequestSuccessRate',
       'InversedDependencyEcosystemExperienceSubmitterPullRequestSuccessRate',
       'SubmitterIsFirstTimeContributor',
       'ControlPullRequestHasCommentByExternalUser',
       'ln(1 + ControlPullRequestLifeTimeInMinutes)',
       'ln(1 + ControlNumberOfCommitsInPullRequest)',
       'ln(1 + ControlIntraProjectPullRequestExperienceOfIntegrator)',
       'ln(1 + IntraProjectSubmitterPullRequestSubmissionCount)',
       'ln(1 + IntraProjectSubmitterPullRequestCommentCount)',
       'ln(1 + EcosystemExperienceSubmitterPullRequestSubmissionCount)',
       'ln(1 + EcosystemExperienceSubmitterPullReques

,ID,Submitter ID,PR Number,IntraProjectSubmitterPullRequestSuccessRate,EcosystemExperienceSubmitterPullRequestSuccessRate,DependencyEcosystemExperienceSubmitterPullRequestSuccessRate,InversedDependencyEcosystemExperienceSubmitterPullRequestSuccessRate,ln(1 + ControlPullRequestLifeTimeInMinutes),ln(1 + ControlNumberOfCommitsInPullRequest),ln(1 + ControlIntraProjectPullRequestExperienceOfIntegrator),...,ln(1 + SharedExperiencePullRequestSubmittedBySubmitterIntegratedByIntegrator),ln(1 + SharedExperiencePullRequestSubmittedByIntegratorIntegratedBySubmitter),ln(1 + SharedExperiencePullRequestSubmittedBySubmitterCommentedOnByIntegrator),ln(1 + SharedExperiencePullRequestSubmittedByIntegratorCommentedOnBySubmitter),ln(1 + SharedExperiencePullRequestDiscussionParticipationByIntegratorAndSubmitter),ln(1 + SharedExperienceIssueSubmittedBySubmitterCommentedOnByIntegrator),ln(1 + SharedExperienceIssueSubmittedByIntegratorCommentedOnBySubmitter),ln(1 + SharedExperienceIssueDiscussionParticipationByIntegratorAndSubmitter),ln(1 + WeightedFirstOrderInDegreeCentrality),ln(1 + WeightedFirstOrderOutDegreeCentrality)
count,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,...,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06,1.224618e+06
mean,4.075076e-01,8.511044e-02,9.717735e-03,5.030779e-01,4.031832e-01,9.883051e-02,5.955435e-02,4.193429e-01,1.184018e-01,2.984268e-01,...,6.189767e-02,1.406902e-02,5.382167e-02,1.615051e-02,4.474016e-02,2.627888e-02,1.639452e-02,3.938143e-02,8.883798e-02,7.433466e-02
std,2.753250e-01,1.332181e-01,2.750959e-02,4.710799e-01,4.523974e-01,2.910963e-01,2.290927e-01,2.256258e-01,7.146971e-02,1.867734e-01,...,1.373575e-01,6.915216e-02,1.309346e-01,6.829825e-02,1.155283e-01,8.868010e-02,7.425770e-02,1.069851e-01,1.050665e-01,9.858844e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.638594e-01,7.794180e-03,5.905393e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.364295e-01,7.525668e-02,1.657954e-01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.784833e-01,2.730277e-02,2.386261e-03,6.842105e-01,0.000000e+00,0.000000e+00,0.000000e+00,4.430626e-01,7.525668e-02,2.971852e-01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.423467e-02,3.550484e-02
75%,6.253565e-01,1.046230e-01,8.231395e-03,1.000000e+00,9.393939e-01,0.000000e+00,0.000000e+00,5.857857e-01,1.505134e-01,4.381478e-01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.367061e-01,1.118951e-01
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [4]:
import copy

default_plot_settings = {
    'edgecolor': 'black',
    'color': "#e69d00"
}

default_plot_settings_2 = copy.deepcopy(default_plot_settings)
default_plot_settings_2['color'] = "#56b3e9"


def __fix_x_label_fontsize(__column):
    # Adjusts x-label's fontsize to fit the text.
    fig = plt.gcf()
    fig_width = fig.get_figwidth()
    xlabel_fontsize = int(fig_width * 100 / len(__column))
    ax = plt.gca()
    orig_fontsize = ax.xaxis.label.get_fontsize()
    ax.xaxis.label.set_fontsize(min(xlabel_fontsize, orig_fontsize))


def create_histogram(_df: pd.DataFrame, _column: str,
                     subfolder_name: str = "",
                     show_without_value = None):
    can_create_feature_histograms = True

    if not can_create_feature_histograms:
        return

    binary_fields = _df.select_dtypes(exclude='number').columns

    print(_column)
    plt.clf()
    entries = _df[_column]

    if _column in binary_fields:
        entries = _df[_column].replace({False: 0, True: 1})
        plt.xticks([0, 1], ['False', 'True'])
        plt.hist(entries, bins=2, **default_plot_settings)
        plt.ylabel('Frequency')
    # elif __column in shown_fields_without_zeroes:
    elif not show_without_value is None:
        _, bins, _ = plt.hist(entries, bins=30, alpha=1,
                              label="All Data", **default_plot_settings)
        ax: plt.Axes = plt.gca()
        ax.set_ylabel("Frequency")
        ax.set_xlabel(_column)
        __fix_x_label_fontsize(_column)

        filtered_data = _df[_column][_df[_column] != show_without_value]
        ax2 = ax.twinx()

        ax2.hist(filtered_data, bins, alpha=0.5,
                 label=f'Excl. {show_without_value}', **default_plot_settings_2)
        ax2.set_ylabel(f"Frequency (excl. x = {show_without_value})")
        ax2.set_zorder(10)
        plt.tight_layout()
    else:
        plt.hist(entries, bins=30, **default_plot_settings)
        plt.ylabel('Frequency')

    plt.xlabel(_column)
    __fix_x_label_fontsize(_column)
    plt.tight_layout()

    output_path = f"{figure_base_path}/distributions/{subfolder_name}/{_column}.png"
    safe_save_fig(output_path)


In [5]:
print(f'{len(df)=}\n')

binary_values = df['ControlIntegratedBySameUser'].value_counts()
print(binary_values)
ratio = binary_values[True] / binary_values[False]
print(f'{ratio=:.03f}\n')

binary_values = df['ControlPullRequestHasComments'].value_counts()
print(binary_values)
ratio = binary_values[True] / binary_values[False]
print(f'{ratio=:.03f}\n')

binary_values = df['ControlHasHashTagInDescription'].value_counts()
print(binary_values)
ratio = binary_values[True] / binary_values[False]
print(f'{ratio=:.03f}\n')

binary_values = df['ControlPullRequestHasCommentByExternalUser'].value_counts()
print(binary_values)
ratio = binary_values[True] / binary_values[False]
print(f'{ratio=:.03f}\n')


binary_values = df['SubmitterIsFirstTimeContributor'].value_counts()
print(binary_values)
ratio = binary_values[True] / binary_values[False]
print(f'{ratio=:.03f}\n')



len(df)=1224618

ControlIntegratedBySameUser
False    701400
True     523218
Name: count, dtype: int64
ratio=0.746

ControlPullRequestHasComments
True     763756
False    460862
Name: count, dtype: int64
ratio=1.657

ControlHasHashTagInDescription
False    822549
True     402069
Name: count, dtype: int64
ratio=0.489

ControlPullRequestHasCommentByExternalUser
False    912555
True     312063
Name: count, dtype: int64
ratio=0.342

SubmitterIsFirstTimeContributor
False    796000
True     428618
Name: count, dtype: int64
ratio=0.538



In [6]:
test_df = df[df['ln(1 + EcosystemExperienceSubmitterPullRequestSubmissionCount)'].gt(0)]
print(f'{len(test_df)=}\n')

binary_values = test_df['ControlIntegratedBySameUser'].value_counts()
print(binary_values)
ratio = binary_values[True] / binary_values[False]
print(f'{ratio=:.03f}\n')

binary_values = test_df['ControlPullRequestHasComments'].value_counts()
print(binary_values)
ratio = binary_values[True] / binary_values[False]
print(f'{ratio=:.03f}\n')

binary_values = test_df['ControlHasHashTagInDescription'].value_counts()
print(binary_values)
ratio = binary_values[True] / binary_values[False]
print(f'{ratio=:.03f}\n')

binary_values = test_df['ControlPullRequestHasCommentByExternalUser'].value_counts()
print(binary_values)
ratio = binary_values[True] / binary_values[False]
print(f'{ratio=:.03f}\n')

binary_values = test_df['SubmitterIsFirstTimeContributor'].value_counts()
print(binary_values)
ratio = binary_values[True] / binary_values[False]
print(f'{ratio=:.03f}\n')


len(test_df)=616125

ControlIntegratedBySameUser
True     314331
False    301794
Name: count, dtype: int64
ratio=1.042

ControlPullRequestHasComments
True     369933
False    246192
Name: count, dtype: int64
ratio=1.503

ControlHasHashTagInDescription
False    406536
True     209589
Name: count, dtype: int64
ratio=0.516

ControlPullRequestHasCommentByExternalUser
False    454535
True     161590
Name: count, dtype: int64
ratio=0.356

SubmitterIsFirstTimeContributor
False    460504
True     155621
Name: count, dtype: int64
ratio=0.338



In [7]:
import scipy


def upscale_id(id: float) -> int:
    return scipy.e ** (id * scipy.log(8.297600e+04)) - 1


In [8]:
print(len(test_df))

binary_values = test_df['ControlPullRequestHasComments'].value_counts()
print(binary_values)
print(binary_values[True] / binary_values[False])

616125
ControlPullRequestHasComments
True     369933
False    246192
Name: count, dtype: int64
1.5026199064145058


In [9]:
test_df = df[['ln(1 + ControlPullRequestLifeTimeInMinutes)']]
test_df.describe()

,ln(1 + ControlPullRequestLifeTimeInMinutes)
count,1.224618e+06
mean,4.193429e-01
std,2.256258e-01
min,0.000000e+00
25%,2.364295e-01
50%,4.430626e-01
75%,5.857857e-01
max,1.000000e+00


In [10]:
instance = df.loc[645]
real_id = upscale_id(instance['ID'])
print(real_id)

515.6005864916907


/tmp/ipykernel_91663/3995248409.py:5: DeprecationWarning: scipy.log is deprecated and will be removed in SciPy 2.0.0, use numpy.lib.scimath.log instead
  return scipy.e ** (1 + id * scipy.log(8.297600e+04))
